# CIFAR10 with Transfer Learning 正式版

In [1]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.noise import GaussianNoise
from keras.layers.normalization import BatchNormalization
from keras.applications import ResNet50, VGG16, InceptionV3
from utils import make_parallel
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time, pickle
from keras.utils import to_categorical
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Prepare Data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## 两种方案

- 严谨的来说，Transfer Learning 指的是拿预训练模型作特征提取器，然后仅训练针对与新数据集的分类器。


- 具体而言，Transfer Learning 有两种方案：
    - 直接将预训练的整个模型拿过来用，训练分类器
    - 自己构建一个和预训练模型一样的模型结构，但是输入尺寸自定义，然后仅导入预训练模型的参数，最后训练分类器。

In [17]:
base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

In [15]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [26]:
x = Input(shape=(32, 32, 3))
y = x
y = Convolution2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='same')(y)

y = Convolution2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='same')(y)

y = Convolution2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='same')(y)

y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='same')(y)

y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = Convolution2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(y)
y = MaxPooling2D(pool_size=2, strides=2, padding='same')(y)

y = GlobalAveragePooling2D()(y)
# y = Dropout(0.5)(y)
# y = Dense(10, activation='softmax')(y)


model = Model(inputs=x, outputs=y)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 8, 8, 128)         0         
__________

In [27]:
model.load_weights('C:\\Users\\Michael\\.keras\\models\\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 8, 8, 128)         0         
__________

In [ ]:
model.

## 1. 放大图片尺寸

- ResNet50 对于输入尺寸有下限要求，因此需要对CIFAR10数据集的图像进行放大才能继续。

### A. 内存方案

In [16]:
X_train_resize = np.zeros((50000, 224, 224, 3), dtype=np.uint8)
for i in range(50000):
    X_train_resize[i] = cv2.resize(X_train[i], (224, 224))

X_test_resize = np.zeros((10000, 224, 224, 3), dtype=np.uint8)
for i in range(10000):
    X_test_resize[i] = cv2.resize(X_test[i], (224, 224))

### B. 生成器方案

- 不能收敛，尚未查明原因

In [19]:
def img_gen(x, y, batch_size, height, width):
    resized = np.zeros((batch_size, height, width, 3), dtype=np.float32)
    while True:
        rand_idx = np.random.choice(range(x.shape[0]), size=batch_size)
        for i in range(batch_size):
            resized[i] = cv2.resize(x[rand_idx[i]], (height, width))
        labels = np.array([y[i] for i in rand_idx])
        yield resized, labels

In [30]:
batch_size = 256
r_height = 100
r_width = 100
train_gen = img_gen(X_train, y_train, batch_size, r_height, r_width)
test_gen = img_gen(X_test, y_test, batch_size, r_height, r_width)

## 2. Transfer Learning - Retrain Classifier

In [21]:
base_model = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False, pooling='avg')

x = base_model.output
x = Dropout(0.5)(x)
y = Dense(10, activation='softmax', kernel_initializer='he_normal')(x)
model = Model(inputs=base_model.input, outputs=y, name='Transfer_Learning')

for layer in base_model.layers:
    layer.trainable = False

model = make_parallel(model, 2)
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [22]:
nb_epoch = 10
batch_size = 256
h = model.fit(x=X_train_resize, 
              y=y_train, 
              batch_size=batch_size, 
              epochs=nb_epoch, 
              validation_data=(X_test_resize, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 223s - loss: 1.4096 - acc: 0.5125 - val_loss: 0.8210 - val_acc: 0.7387
Epoch 2/10
50000/50000 [==============================] - 221s - loss: 0.9153 - acc: 0.6864 - val_loss: 0.7193 - val_acc: 0.7620
Epoch 3/10
50000/50000 [==============================] - 220s - loss: 0.8117 - acc: 0.7207 - val_loss: 0.6658 - val_acc: 0.7778
Epoch 4/10
50000/50000 [==============================] - 221s - loss: 0.7571 - acc: 0.7400 - val_loss: 0.6410 - val_acc: 0.7843
Epoch 5/10
50000/50000 [==============================] - 221s - loss: 0.7274 - acc: 0.7511 - val_loss: 0.6093 - val_acc: 0.7934
Epoch 6/10
50000/50000 [==============================] - 221s - loss: 0.7077 - acc: 0.7547 - val_loss: 0.6009 - val_acc: 0.7953
Epoch 7/10
50000/50000 [==============================] - 221s - loss: 0.6870 - acc: 0.7628 - val_loss: 0.5869 - val_acc: 0.8001
Epoch 8/10
50000/50000 [=======================